In [ ]:
!pip install requests
!pip install beautifulsoup4
!pip install spacy
import nltk
nltk.download('all')
from	nltk	import	word_tokenize
from nltk import re
palabra = "eth"

In [ ]:
import requests
from textblob import TextBlob
from bs4 import BeautifulSoup

def scrape_url(url : str):
  polaridadTot = 0
  texto_com = ""
  response = requests.get(url)
  soup = BeautifulSoup(response.content, "html.parser")
  paragraphs = soup.find_all('p')
  titulo = soup.title.text.strip()
  texto_com = " ".join(paragraph.text for paragraph in paragraphs)
  polaridad = TextBlob(texto_com).sentiment.polarity
 # if polaridad > 0:
  #  print(titulo + " positivo ")
  #elif polaridad < 0:
   # print(titulo + " negativo ")
  #else:
   # print(titulo + " neutro ")
  #print(polaridad)
  return {"title": titulo, "body": texto_com, "url": url}

In [ ]:
#google scraping
from googlesearch import search
#help(search)
def scrape_google():
	l = []
	results = search(palabra,stop=5,tpe="nws")
	for r in results:
		l.append(scrape_url(r))
	return l

In [ ]:
#bing scraping
import requests
import lxml
from bs4 import BeautifulSoup

def scrape_bing():
  l=[]
  target_url="https://www.bing.com/news/search?q=" + palabra
  headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}
  resp=requests.get(target_url,headers=headers)
  soup = BeautifulSoup(resp.text, "lxml")
  for result in soup.select('.card-with-cluster')[:5]:
    bingUrl = result.select_one('.title')['href']
    l.append(scrape_url(bingUrl))
  return l

In [ ]:
#cointelegraph scraping
import requests
import lxml
from bs4 import BeautifulSoup

def scrape_cointelegraph():
  l = []
  url = 'https://cointelegraph.com/tags/ethereum'
  headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}
  response = requests.get(url,headers=headers)
  soup = BeautifulSoup(response.content, 'html.parser')
  news_elements = soup.find_all(class_='post-card-inline__title-link')[:2]
  for news_element in news_elements:
    link = news_element['href']
    l.append(scrape_url("https://cointelegraph.com" + link))
  return l

In [ ]:
#agarrar palabras con sentimiento
from textblob import TextBlob
def filtrarPorPolaridad(listaPal):
  l = []
  for palabra in listaPal:
    if TextBlob(palabra).sentiment.polarity != 0:
      l.append(palabra)
  return(l)

In [ ]:
#calcular frecuencia por palabra
def listaPalabrasDicFrec(listaPal):
  frecuenciaPalab = [listaPal.count(p) for p in listaPalabras]
  return dict(list(zip(listaPalabras,frecuenciaPalab)))

In [ ]:
#ordenar palabras por frequencia
def ordenaDicFrec(dicfrec):
    aux = [(dicfrec[key], key) for key in dicfrec]
    aux.sort()
    aux.reverse()
    return aux

In [ ]:
#Frequency
def palabrasPrin(palabras,num):
  listaFreq = listaPalabrasDicFrec(palabras)
  listaFreqOrd = ordenaDicFrec(listaFreq)
  listaPrin = listaFreqOrd[:num]
  return listaPrin

In [ ]:
#TiemStamp
import calendar
import time
def time_Stamp():
  time_stam = time.time()
  return time_stam

In [ ]:
#Ner
import spacy
def devolverNer(textoTot):
  nlp = spacy.load("en_core_web_sm")
  ner_categories = ["PERSON","ORG","GPE","PRODUCT"]
  listaNer = []
  doc = nlp(textoTot)
  entidades = []
  for ent in doc.ents:
    if ent.label_ in ner_categories:
      entidades.append((ent.text,ent.label_))
  for entity,category in entidades:
    listaNer.append(entity)
  return set(listaNer)

In [ ]:
#Sentimiento
from textblob import TextBlob
def sentimientoTexto(texto):
  polaridad = TextBlob(texto).sentiment.polarity
  if polaridad > 0:
    return "positivo"
  elif polaridad > 0:
    return "negativo"
  else:
    return "neutro"

In [ ]:
#Base de Datos
from csv import writer

def cargarBaseDeDatos(sent,time,palPrinc,ner):
  list_data = [sent,time,palPrinc,ner]
  with open("BaseDeDatos.csv", "a", newline="") as f_object:
      writer_object = writer(f_object)
      writer_object.writerow(list_data)
      f_object.close()

In [ ]:
#crear base de datos
cargarBaseDeDatos("Sentimiento","Time Stamp","Palabras Principales","Ner")

In [ ]:
def tokenizarTexto(texto):
  palabrasTotal = [word	for	word	in	word_tokenize(texto.lower())	if	re.search("\w",	word)]
  return palabrasTotal

In [ ]:
listaTotal = scrape_bing() + scrape_google() + scrape_cointelegraph()
listaPalabras = []
textoTotal = " ".join(noticia["body"] for noticia in listaTotal)
for pagina in listaTotal:
  listaPalabras = listaPalabras + word_tokenize(textoTotal.lower())
listaFiltrada = filtrarPorPolaridad(listaPalabras)
palPrin = palabrasPrin(listaFiltrada,10)
sentimiento = sentimientoTexto(textoTotal)
timeStamp = time_Stamp()
palabrasNer = devolverNer(textoTotal)
cargarBaseDeDatos(sentimiento,timeStamp,palPrin,palabrasNer)

